California, Oregon, Arizona, Texas, Colorado, Illinois, Florida, Georgia, North Carolina, Pennsylvania

Ohio, Massachusetts, Michigan, Minnesota, Wisconsin, New York, Utah, Indiana, Louisiana, Oklahoma, Tennessee

In [ ]:
! pip install pmdarima==1.8.0

     |████████████████████████████████| 1.5MB 6.0MB/s 
     |████████████████████████████████| 9.5MB 16.5MB/s 
     |████████████████████████████████| 2.1MB 39.7MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: Cython 0.29.22
    Uninstalling Cython-0.29.22:
      Successfully uninstalled Cython-0.29.22


In [ ]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt

In [ ]:
#load vaccination data and population data 
data = pd.read_csv('/content/us_state_vaccinations.csv')
population_data = pd.read_csv('/content/csvData.csv')

In [ ]:
#get population for desired state
population = population_data[['State', 'Pop']]
pop_state = population[population['State'] == 'New York']
state_pop = pop_state['Pop'].iloc[0]
state_pop

19300000

In [ ]:
#get desired vaccination data for certain state, convert date to standard datetime, and set date as index
california = data[data['location'] == 'New York State']
california['date'] = pd.to_datetime(california['date'])
california.set_index('date', inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#take number of people who are fully vaccinated
california2 = california[['people_fully_vaccinated']]

#drop NaN, change data type to integer 
california3 = california2.dropna()
california3['people_fully_vaccinated'] = california3['people_fully_vaccinated'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#build auto.arima model
y = california3['people_fully_vaccinated']
arima_model2 = auto_arima(y)
#arima_model2.summary()

In [ ]:
#make prediction 
import pandas as pd
from datetime import datetime


datelist = pd.DataFrame(pd.date_range(start="2021-04-15", periods=150).tolist())
prediction2 = pd.DataFrame(arima_model2.predict(n_periods = 150))
prediction2.columns = ['predicted total num of vaccination']
prediction2['predicted total num of vaccination'] = round(prediction2['predicted total num of vaccination']).astype(int)

df_final = pd.concat([datelist, prediction2], axis=1)
df_final.columns = ['date','people_fully_vaccinated']
df_final.set_index('date', inplace = True)

final = pd.concat([california3,df_final])
final['people_fully_vaccinated'] = final['people_fully_vaccinated'].astype(int)
final['percentage'] = final['people_fully_vaccinated'] / state_pop

#add state name
state = pd.DataFrame(['New York'] * len(final))
final.reset_index(drop=False, inplace=True)
state.reset_index(drop=True, inplace=True)

df = pd.concat( [final, state], axis=1) 
df.columns = ['Date','people_fully_vaccinated','Percentage','State']

df

,Date,people_fully_vaccinated,Percentage,State
0,2021-01-12,36422,0.001887,New York
1,2021-01-13,44512,0.002306,New York
2,2021-01-15,73523,0.003809,New York
3,2021-01-19,102187,0.005295,New York
4,2021-01-20,106821,0.005535,New York
...,...,...,...,...
233,2021-09-07,21854035,1.132333,New York
234,2021-09-08,21968530,1.138266,New York
235,2021-09-09,22083024,1.144198,New York
236,2021-09-10,22197518,1.150130,New York


In [ ]:
#save to csv file
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/Shareddrives/The Data Collective/Vaccination Forecast/forecast_file_with_state/NewYork.csv')


Mounted at /content/drive


Read all csv file and compile into a single file

In [ ]:
import pandas as pd
import glob

path = r'/content/drive/Shareddrives/The Data Collective/Vaccination Forecast/forecast_file_with_state' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame = frame.drop(columns=['Unnamed: 0'])
frame

,Date,people_fully_vaccinated,Percentage,State
0,2021-01-12,10591,0.000827,Pennsylvania
1,2021-01-13,18051,0.001410,Pennsylvania
2,2021-01-15,66941,0.005228,Pennsylvania
3,2021-01-19,79526,0.006211,Pennsylvania
4,2021-01-20,85290,0.006661,Pennsylvania
...,...,...,...,...
4991,2021-09-07,21854035,1.132333,New York
4992,2021-09-08,21968530,1.138266,New York
4993,2021-09-09,22083024,1.144198,New York
4994,2021-09-10,22197518,1.150130,New York


In [ ]:
#save to csv file
from google.colab import drive
drive.mount('/content/drive')
frame.to_csv('/content/drive/Shareddrives/The Data Collective/Dataset/20state.csv', index = False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
